In [70]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [71]:
pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [72]:
pip install tabulate

Note: you may need to restart the kernel to use updated packages.


In [73]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup


driver = webdriver.Chrome()
driver.get("https://boulder.co.publicsearch.us/")
#print(driver.page_source)

search_box = driver.find_element(By.CSS_SELECTOR, '[data-testid="searchInputBox"]')

# AT THE MOMENT, I'M USING MY DAD'S NAME
search_box.send_keys("Sanjay Shah")
search_box.send_keys(Keys.RETURN)

# wait for the page to load after submitting the search
# driver.implicitly_wait(5)

# Wait for the page to load after submitting the search (adjust wait time if necessary)
WebDriverWait(driver, 5).until(
    EC.presence_of_element_located((By.TAG_NAME, 'tfoot'))  # Wait for the <tfoot> tag to appear
)

soup2 = BeautifulSoup(driver.page_source, "html.parser")
# print(soup2.prettify())
# driver.quit()

## various output formatting styles

### list view

In [74]:
# Find the table header row <tr> inside the <tfoot>
header_row = soup2.find('tfoot').find('tr')
    
# Extract all the <th> elements (headers) from the row
headers = header_row.find_all('th')

# Print the table headers
print("Headers:")
for header in headers:
    print(f"• {header.get_text(strip=True)}")

Headers:
• 
• 
• 
• Reception Number
• Book
• Page
• Doc Type
• Grantor
• Grantee
• Recorded Date
• Legal Description
• References


### Rows of data

In [75]:
# Find all the table rows <tr> inside the <tbody> (this should be where the rows are located)
rows = soup2.find_all('tr', class_='')

# Print all the rows under the table headers
print("Rows:")
for row in rows:
    cols = row.find_all('td')  # Extract all <td> elements (columns)
    
    if len(cols) > 1:  # Ensure it's a valid row with data (ignores empty rows)
        row_data = [col.get_text(strip=True) for col in cols]  # Extract and clean up the text
        print(row_data)

# Close the driver
driver.quit()

Rows:
['', '', '', '02469067', 'N/A', 'N/A', 'DEED OF TRUST', 'SHAHBHAVI', 'WELLS FARGO HOME MORTGAGE', '7/11/2003', 'L 14 B 7 DODD ESTATES #1', '02978162']
['', '', '', '02343160', 'N/A', 'N/A', 'DEED OF TRUST', 'SHAHBAHVI', 'JOHN LAING MORTGAGE', '10/14/2002', 'L 14 B 7 DODD ESTATES #1', '02343161']
['', '', '', '02975443', 'N/A', 'N/A', 'DEED OF TRUST', 'SHAHBHAVI', 'ELEVATIONS CREDIT UNION', '1/27/2009', 'Subdivision-  Name: DODD ESTATES 1ST Lot: 14 Block: 7', '03152791']
['', '', '', '03151268', 'N/A', 'N/A', 'DEED OF TRUST', 'SHAHBHAVI', 'INTERBANK MORTGAGE COMPANY', '5/31/2011', 'Subdivision-  Name: DODD ESTATES 1ST Lot: 14 Block: 7', '03216351']
['', '', '', '03244160', 'N/A', 'N/A', 'DEED OF TRUST', 'SHAHBHAVI', 'PROVIDENT FUNDING ASSOCIATES LP', '8/13/2012', 'Subdivision-  Name: DODD ESTATES 1ST Lot: 14 Block: 7', '03756314']
['', '', '', '03201248', 'N/A', 'N/A', 'DEED OF TRUST', 'SHAHBHAVI', 'MORTGAGE ELECTRONIC REGISTRATION SYSTEMS INC NOMINEE', '2/7/2012', 'Subdivision-  

### table layout using tabulate

In [76]:
from tabulate import tabulate

# Find the table header row <tr> inside the <tfoot>
header_row = soup2.find('tfoot').find('tr')

# Extract all the <th> elements (headers) from the row
headers = header_row.find_all('th')

# Create a list of headers
header_titles = [header.get_text(strip=True) for header in headers]

# Define the specific columns you want to keep (based on the headers you provided)
columns_to_keep = ["Reception Number", "Doc Type", "Grantor", "Grantee", "Recorded Date", "Legal Description", "References"]

# Find all the table rows <tr> inside the <tbody>
rows = soup2.find_all('tr', class_='')

# Prepare the rows to match the number of columns and clean up the data
table_rows = []
for row in rows:
    cols = row.find_all('td')  # Extract all <td> elements (columns)
    
    if len(cols) > 1:  # Ensure it's a valid row with data (ignores empty rows)
        row_data = [col.get_text(strip=True) for col in cols]
        
        # Create a filtered row with only the specified columns
        filtered_row = []
        for col_name in columns_to_keep:
            if col_name in header_titles:
                col_index = header_titles.index(col_name)
                filtered_row.append(row_data[col_index])
        
        if filtered_row:  # Add only non-empty rows
            table_rows.append(filtered_row)

# Print the table in a clean format using tabulate
# print("Table Data:")
# print(tabulate(table_rows, headers=header_titles, tablefmt="fancy_grid"))
print("Filtered Table Data:")
print(tabulate(table_rows, headers=columns_to_keep, tablefmt="fancy_grid"))

driver.quit()

Filtered Table Data:
╒════════════════════╤═════════════════════════════╤════════════════════════════════════════╤══════════════════════════════════════════════════════╤═════════════════╤═══════════════════════════════════════════════════════╤══════════════╕
│   Reception Number │ Doc Type                    │ Grantor                                │ Grantee                                              │ Recorded Date   │ Legal Description                                     │ References   │
╞════════════════════╪═════════════════════════════╪════════════════════════════════════════╪══════════════════════════════════════════════════════╪═════════════════╪═══════════════════════════════════════════════════════╪══════════════╡
│           02469067 │ DEED OF TRUST               │ SHAHBHAVI                              │ WELLS FARGO HOME MORTGAGE                            │ 7/11/2003       │ L 14 B 7 DODD ESTATES #1                              │ 02978162     │
├────────────────────┼─────